In [20]:
# Position & Portfolio

# Position = combination of some specific stock & its quantity
# Portfolio = list of position types

In [21]:
import requests 
from bs4 import BeautifulSoup

from dataclasses import dataclass

@dataclass
class Stock:
    ticker: str
    exchange: str
    price: float=0
    currency: str='USD'
    usd_price: float=0

    def __post_init__(self):                                    
        price_info=get_price_info(self.ticker,self.exchange)    
        if price_info['ticker']==self.ticker:                   
            self.price=price_info['price']
            self.currency=price_info['currency']
            self.usd_price=price_info['usd_price']


@dataclass
class Position:
    stock: Stock
    quantity: int


@dataclass
class Portfolio:
    positions: list[Position]

    def get_total_value(self):      # self is automatically populated, because get_totoal_val is an instance method & all instance methods in python have 'self' as their first argument
        total_value=0
        for position in self.positions:
            total_value+=position.quantity*position.stock.usd_price
        return total_value



def get_fx_to_usd(currency):
    fx_url=f'https://www.google.com/finance/quote/{currency}-USD' 
    response=requests.get(fx_url)                                    
    soup=BeautifulSoup(response.content,"html.parser")            
    fx_rate=soup.find('div',attrs={'data-last-price':True})     
    fx=float(fx_rate['data-last-price'])
    return fx


def get_price_info(ticker,exchange):                                    
    url=f'https://www.google.com/finance/quote/{ticker}:{exchange}'     
    response=requests.get(url)                                       
    soup=BeautifulSoup(response.content,"html.parser")               
    price_div=soup.find('div',attrs={'data-last-price':True})        
    price=float(price_div['data-last-price'])                        
    currency=price_div['data-currency-code']                         
    
    usd_price=price
    if(currency!='USD'):                                             
        fx=get_fx_to_usd(currency)
        usd_price=round(price*fx,2)
    
    return {
        'ticker':ticker,
        'exchange':exchange,
        'price':price,
        'currency':currency,
        'usd_price':usd_price
    }

In [22]:
shop=Stock('SHOP','TSE')
tsla=Stock('TSLA','NASDAQ')
aapl=Stock('AAPL','NASDAQ')
amzn=Stock('AMZN','NASDAQ')
amzn

Stock(ticker='AMZN', exchange='NASDAQ', price=133.56, currency='USD', usd_price=133.56)

In [23]:
Position(aapl,10)
# consists of an instance of Stock (user-defined), also has a quantity

Position(stock=Stock(ticker='AAPL', exchange='NASDAQ', price=193.99, currency='USD', usd_price=193.99), quantity=10)

In [24]:
portfolio=Portfolio([Position(shop,10), Position(tsla,5), Position(aapl,7), Position(amzn,10)])
portfolio

Portfolio(positions=[Position(stock=Stock(ticker='SHOP', exchange='TSE', price=90.63, currency='CAD', usd_price=68.73), quantity=10), Position(stock=Stock(ticker='TSLA', exchange='NASDAQ', price=290.38, currency='USD', usd_price=290.38), quantity=5), Position(stock=Stock(ticker='AAPL', exchange='NASDAQ', price=193.99, currency='USD', usd_price=193.99), quantity=7), Position(stock=Stock(ticker='AMZN', exchange='NASDAQ', price=133.56, currency='USD', usd_price=133.56), quantity=10)])

In [25]:
portfolio.get_total_value()

4832.73